In [1]:
# ! pip install transformers
import torch
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

import torchsummary as summary
from tqdm import tqdm

from torch import nn
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os
import base64
import gc

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def create_onedrive_directdownload(onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

In [3]:
onedrive_link='https://1drv.ms/u/s!AoiE7xOoBAsngsgmC-PI9ray-gfELQ?e=UtffII'
onedrive_direct_link=create_onedrive_directdownload(onedrive_link)
onedrive_direct_link
df=pd.read_csv(onedrive_direct_link)
df.drop('Unnamed: 0',axis=1,inplace=True)
df=df[['news_content','rule_labels_comb']]
df=df[~df['rule_labels_comb'].isna()]
df=df[(df['rule_labels_comb'].apply(len)!=2)]
df = df.reset_index()
del df['index']
df

,news_content,rule_labels_comb
0,The article discusses how M.M. Warburg may hav...,['legal']
1,The Cologne Prosecutor's Office accuses German...,"['collaboration', 'legal']"
2,Deutsche Bank AG has acquired an additional 49...,['collaboration']
3,Mozambique has reached a negotiated agreement ...,"['collaboration', 'legal']"
4,The UBS APAC Sustainable Finance Conference 20...,"['personnel', 'legal']"
...,...,...
2589,Arbeiten bei HSBC Trinkaus & Burkhardt GmbH in...,['product']
2590,HSBC muss MillionenStrafe wegen WhatsAppNutzun...,['personnel']
2591,Alle offenen Sozialkompetent Jobs bei Bethmann...,['personnel']
2592,Citigroup Inc. Deutsche Bank AG HSBC Holdings ...,['legal']


In [4]:
from ast import literal_eval
df['rule_labels_comb'] = df['rule_labels_comb'].apply(literal_eval)
df['target']='[0.0,0.0,0.0,0.0]'
df['target'] = df['target'].apply(literal_eval)

df['target'].iloc[0][0]

0.0

In [5]:
for index, value in df['rule_labels_comb'].iteritems():
    class_dic={'personnel':0,'product':1,'collaboration':2,'legal':3}
    val=[class_dic.get(i, i) for i in value]
    df['rule_labels_comb'].iloc[index]=val
    y=0
    for x in df['rule_labels_comb'].iloc[index]:
        df['target'].iloc[index][x]=1.0
        y=y+1
df['rule_labels_comb']

0          [3]
1       [2, 3]
2          [2]
3       [2, 3]
4       [0, 3]
         ...  
2589       [1]
2590       [0]
2591       [0]
2592       [3]
2593       [3]
Name: rule_labels_comb, Length: 2594, dtype: object

In [6]:
df['target']

0       [0.0, 0.0, 0.0, 1.0]
1       [0.0, 0.0, 1.0, 1.0]
2       [0.0, 0.0, 1.0, 0.0]
3       [0.0, 0.0, 1.0, 1.0]
4       [1.0, 0.0, 0.0, 1.0]
                ...         
2589    [0.0, 1.0, 0.0, 0.0]
2590    [1.0, 0.0, 0.0, 0.0]
2591    [1.0, 0.0, 0.0, 0.0]
2592    [0.0, 0.0, 0.0, 1.0]
2593    [0.0, 0.0, 0.0, 1.0]
Name: target, Length: 2594, dtype: object

In [7]:
torch.cuda.empty_cache()

In [8]:
gc.collect()

66

In [9]:
# import data from gdrive
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('BERT_data.csv')\ndf=df[~(df['content']=='nan')]\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [10]:
# change range of labels, minimum should be zero
df['rule_labels_comb']=df['rule_labels_comb'].astype(str)
df

,news_content,rule_labels_comb,target
0,The article discusses how M.M. Warburg may hav...,[3],"[0.0, 0.0, 0.0, 1.0]"
1,The Cologne Prosecutor's Office accuses German...,"[2, 3]","[0.0, 0.0, 1.0, 1.0]"
2,Deutsche Bank AG has acquired an additional 49...,[2],"[0.0, 0.0, 1.0, 0.0]"
3,Mozambique has reached a negotiated agreement ...,"[2, 3]","[0.0, 0.0, 1.0, 1.0]"
4,The UBS APAC Sustainable Finance Conference 20...,"[0, 3]","[1.0, 0.0, 0.0, 1.0]"
...,...,...,...
2589,Arbeiten bei HSBC Trinkaus & Burkhardt GmbH in...,[1],"[0.0, 1.0, 0.0, 0.0]"
2590,HSBC muss MillionenStrafe wegen WhatsAppNutzun...,[0],"[1.0, 0.0, 0.0, 0.0]"
2591,Alle offenen Sozialkompetent Jobs bei Bethmann...,[0],"[1.0, 0.0, 0.0, 0.0]"
2592,Citigroup Inc. Deutsche Bank AG HSBC Holdings ...,[3],"[0.0, 0.0, 0.0, 1.0]"


In [11]:
# additional filtering to balance classes
'''
df_3=df[(df['result']==3) & (df['content'].str.len()<350)]
df=df[~(df['result']==3)]
df=pd.concat([df,df_3])
df
'''

"\ndf_3=df[(df['result']==3) & (df['content'].str.len()<350)]\ndf=df[~(df['result']==3)]\ndf=pd.concat([df,df_3])\ndf\n"

In [12]:
# import BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['target']
        self.text=[tokenizer(text,padding='max_length',truncation=True,return_tensors="pt") for text in df['news_content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [13]:
# train test split
X_train, X_test= train_test_split(df[['news_content','target']], test_size=0.25)

In [14]:
df_train=X_train
df_val=X_test
df_test=0

In [15]:
df_train

,news_content,target
552,The article discusses a research analysis and ...,"[0.0, 1.0, 1.0, 1.0]"
1746,A Berlin-based family office has advertised fo...,"[1.0, 0.0, 0.0, 1.0]"
4,The UBS APAC Sustainable Finance Conference 20...,"[1.0, 0.0, 0.0, 1.0]"
2248,The German bank M.M. Warburg is reportedly exp...,"[1.0, 0.0, 1.0, 1.0]"
1004,The Versorgungsanstalt des Bundes und der LÃ¤n...,"[1.0, 0.0, 1.0, 1.0]"
...,...,...
2474,The Swiss government will not launch an invest...,"[1.0, 0.0, 0.0, 1.0]"
2436,Group of experts on banking stability now unde...,"[0.0, 0.0, 0.0, 1.0]"
36,The article discusses the connection between D...,"[1.0, 0.0, 1.0, 1.0]"
1175,2023 Archegos: FINMA schliesst Verfahren gegen...,"[0.0, 1.0, 1.0, 1.0]"


In [16]:
# BERT classifier architecture, with 7 output classes
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()
        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [17]:
# change runtype to GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [18]:
# hyperparameters
EPOCHS = 50
model = BertClassifier()
LR = 0.000001

In [19]:
df_train=df_train.reset_index()
df_train.drop('index',axis=1,inplace=True)
df_train

,news_content,target
0,The article discusses a research analysis and ...,"[0.0, 1.0, 1.0, 1.0]"
1,A Berlin-based family office has advertised fo...,"[1.0, 0.0, 0.0, 1.0]"
2,The UBS APAC Sustainable Finance Conference 20...,"[1.0, 0.0, 0.0, 1.0]"
3,The German bank M.M. Warburg is reportedly exp...,"[1.0, 0.0, 1.0, 1.0]"
4,The Versorgungsanstalt des Bundes und der LÃ¤n...,"[1.0, 0.0, 1.0, 1.0]"
...,...,...
1940,The Swiss government will not launch an invest...,"[1.0, 0.0, 0.0, 1.0]"
1941,Group of experts on banking stability now unde...,"[0.0, 0.0, 0.0, 1.0]"
1942,The article discusses the connection between D...,"[1.0, 0.0, 1.0, 1.0]"
1943,2023 Archegos: FINMA schliesst Verfahren gegen...,"[0.0, 1.0, 1.0, 1.0]"


In [20]:
# display BERT layers
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

201

In [21]:
# freeze first 8 layers 
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [22]:
# change datatypes of input data
df_train['news_content']=df_train['news_content'].astype(str)

df_val['news_content']=df_val['news_content'].astype(str)
df_train['news_content'].iloc[0]

'The article discusses a research analysis and presentation conducted by ODDO BHF in Frankfurt, Germany, aimed at acquiring potential clients. The analysis and presentation took place over seven months, from February 2023 to the present. The purpose of this initiative is to attract new clients and showcase the expertise and services offered by ODDO BHF. Frankfurt, located in the Hesse region of Germany, serves as the headquarters for this research and analysis project.'

In [24]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    
    # mini batching
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=30)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    
    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device) # to cuda GPU
                mask = train_input['attention_mask'].to(device) # attention mask
                input_id = train_input['input_ids'].squeeze(1).to(device)
                
                l1_loss=0
                
                # for L1 regularization
                a=0
                reg_loss = 0
                for param in model.parameters():
                    a=a+1
                    if a >=201-68:
                        reg_loss += torch.norm(param, 1) 
                
                factor = 0.00001 #lambda for L1 regularization
                l1_loss=factor * reg_loss # L1 loss
                
                # model output
                output = model(input_id, mask)
                
                # loss value
                batch_loss = criterion(output, train_label) + l1_loss
                total_loss_train += batch_loss.item() 
                '''
                # train accuracy 
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                '''
                # backpropogation
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0
            
            # for validation accuracy
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    
                    # validation output
                    output = model(input_id, mask)
                    
                    # validation loss value
                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    '''
                    # validation accuracy
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
'''
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')



train(model, df_train, df_val, LR, EPOCHS)

  9%|██████▉                                                                          | 83/973 [00:08<01:30,  9.85it/s]


KeyboardInterrupt: 

In [272]:
# incase test data is used
'''
%%timeit -n 1 -r 1
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)
'''

'\n%%timeit -n 1 -r 1\ndef evaluate(model, test_data):\n\n    test = Dataset(test_data)\n\n    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)\n\n    use_cuda = torch.cuda.is_available()\n    device = torch.device("cuda" if use_cuda else "cpu")\n\n    if use_cuda:\n\n        model = model.cuda()\n\n    total_acc_test = 0\n    with torch.no_grad():\n\n        for test_input, test_label in test_dataloader:\n\n              test_label = test_label.to(device)\n              mask = test_input[\'attention_mask\'].to(device)\n              input_id = test_input[\'input_ids\'].squeeze(1).to(device)\n\n              output = model(input_id, mask)\n\n              acc = (output.argmax(dim=1) == test_label).sum().item()\n              total_acc_test += acc\n\n    print(f\'Test Accuracy: {total_acc_test / len(test_data): .3f}\')\n\nevaluate(model, df_test)\n'

In [ ]:
# save model file
torch.save(model.state_dict(), 'BERT_model_freeze.pth')